# APIM ❤️ OpenAI

## Prompt flow lab
![flow](../../images/prompt-flow.gif)

Playground to try the [Azure AI Studio Prompt Flow](https://learn.microsoft.com/en-us/azure/ai-studio/how-to/prompt-flow) with Azure API Management. The Prompt Flow OpenAI connection will be facilitated by APIM, enabling load balancing, token counting, and other features. The Prompt Flow will run in an Azure Container App and will be accessed through APIM.

Prompt flow is a development tool designed to streamline the entire development cycle of AI applications powered by Large Language Models (LLMs). Prompt flow provides a comprehensive solution that simplifies the process of prototyping, experimenting, iterating, and deploying your AI applications.  
Prompt flow is available independently as an open-source project on [GitHub](https://github.com/microsoft/promptflow).

### Result
![result](result.png)

### TOC
- [0️⃣ Initialize notebook variables](#0)
- [1️⃣ Create the Azure Resource Group](#1)
- [2️⃣ Create deployment using 🦾 Bicep](#2)
- [3️⃣ Get the deployment outputs](#3)
- [4️⃣ Enable Prompt Flow tracing (optional)](#4)
- [5️⃣ Update the connection with APIM Gateway URL and APIM Subscription Key](#5)
- [🧪 Test the Flow locally](#localtest)
- [6️⃣ Create the Flow in the AI Studio Project (optional)](#6)
- [7️⃣ Build the Flow in the Docker format](#7)
- [8️⃣ Deploy the Flow to an Azure Container App](#8)
- [🧪 Test the Flow through APIM](#requests)
- [🔍 Analyze Application Insights requests](#portal)
- [🗑️ Clean up resources](#clean)

### Prerequisites
- [Python 3.8 or later version](https://www.python.org/) installed
- [Prompt flow](https://microsoft.github.io/promptflow/how-to-guides/installation/index.html#install-prompt-flow) installed
- [VS Code](https://code.visualstudio.com/) installed with the [Jupyter notebook extension](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter) enabled
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli) installed
- [An Azure Subscription](https://azure.microsoft.com/en-us/free/) with Contributor permissions
- [Access granted to Azure OpenAI](https://aka.ms/oai/access)
- [Sign in to Azure with Azure CLI](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)

<a id='0'></a>
### 0️⃣ Initialize notebook variables

- Resources will be suffixed by a unique string based on your subscription id
- Adjust the location parameters according your preferences and on the [product availability by Azure region.](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management) 
- Adjust the OpenAI model and version according the [availability by region.](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models) 

In [1]:
import os
import json
import datetime
import requests

lab_prefix="aip-pevo1-"
tool_prefix="st-pevo1-"
deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"{lab_prefix}{deployment_name}" # change the name to match your naming style

resource_group_location = "westeurope"
apim_resource_name = "apim"
apim_resource_location = "westeurope"
#apim_resource_sku = "Basicv2"
apim_resource_sku = "Consumption" 


openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"} ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ ]

log_analytics_name = "workspace"
app_insights_name = 'insights'

ai_studio_hub_name = 'hub'
ai_studio_project_name = 'project'
storage_account_name = 'storage'
keyvault_name = 'keyvault'
container_registry_name = 'registry'

containerapp_env_name = 'acaenv'
containerapp_name = 'aca'

flow_name = 'basic-chat' # the local folder must have the same name
print("✅ Set Variables done ⌚ ", datetime.datetime.now().isoformat())



✅ Set Variables done ⌚  2024-10-25T11:33:57.752156


<a id='1'></a>
### 1️⃣ Create the Azure Resource Group
All resources deployed in this lab will be created in the specified resource group. Skip this step if you want to use an existing resource group.

In [2]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

✅ Azure Resource Group  aip-pevo1-prompt-flow  created ⌚  11:34:10.333277


<a id='2'></a>
### 2️⃣ Create deployment using 🦾 Bicep

This lab uses [Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep) to declarative define all the resources that will be deployed. Change the parameters or the [main.bicep](main.bicep) directly to try different configurations. 

In [3]:
if len(openai_resources) > 0:
    backend_id = openai_backend_pool if len(openai_resources) > 1 else openai_resources[0].get("name")

with open("policy.xml", 'r') as policy_xml_file:
    policy_template_xml = policy_xml_file.read()
    policy_xml = policy_template_xml.replace("{backend-id}", backend_id)
    policy_xml_file.close()
open("policy.xml", 'w').write(policy_xml)

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku},
    "logAnalyticsName": { "value": log_analytics_name },
    "applicationInsightsName": { "value": app_insights_name },
    "aiStudioHubName": { "value": ai_studio_hub_name },
    "aiStudioProjectName": { "value": ai_studio_project_name },
    "storageAccountName": { "value": storage_account_name }, 
    "keyVaultName": { "value": keyvault_name },
    "containerRegistryName": { "value": container_registry_name },
    "containerAppEnvName": { "value": containerapp_env_name },
    "containerAppName": { "value": containerapp_name },
    "flowName": { "value": flow_name }
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

open("policy.xml", 'w').write(policy_template_xml)


{
  "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aip-pevo1-prompt-flow/providers/Microsoft.Resources/deployments/prompt-flow",
  "location": null,
  "name": "prompt-flow",
  "properties": {
    "correlationId": "35312160-fd1f-43c8-ac94-687c54443f1a",
    "debugSetting": null,
    "dependencies": [
      {
        "dependsOn": [
          {
            "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aip-pevo1-prompt-flow/providers/Microsoft.Storage/storageAccounts/storage4hx6wwntb767o/blobServices/default",
            "resourceGroup": "aip-pevo1-prompt-flow",
            "resourceName": "storage4hx6wwntb767o/default",
            "resourceType": "Microsoft.Storage/storageAccounts/blobServices"
          }
        ],
        "id": "/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aip-pevo1-prompt-flow/providers/Microsoft.Storage/storageAccounts/storage4hx6wwntb767o/blobServices/default/containers/default",
        


c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\prompt-flow\main.bicep(340,3) : Warning BCP035: The specified "object" declaration is missing the following required properties: "protocol", "url". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagnostics#BCP035]

c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\labs\prompt-flow\main.bicep(405,5) : Warning BCP037: The property "CustomMetricsOptedInType" is not allowed on objects of type "ApplicationInsightsComponentProperties". Permissible properties include "DisableIpMasking", "DisableLocalAuth", "Flow_Type", "ForceCustomerStorageForProfiler", "HockeyAppId", "ImmediatePurgeDataOn30Days", "IngestionMode", "publicNetworkAccessForIngestion", "publicNetworkAccessForQuery", "Request_Source", "RetentionInDays", "SamplingPercentage". If this is a resource type definition inaccuracy, report it using https://aka.ms/bicep-type-issues. [https://aka.ms/bicep/core-diagno

618

<a id='3'></a>
### 3️⃣ Get the deployment outputs


In [4]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.logAnalyticsWorkspaceId.value -o tsv
workspace_id = deployment_stdout.n
print("👉🏻 Workspace ID: ", workspace_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.applicationInsightsAppId.value -o tsv
app_id = deployment_stdout.n
print("👉🏻 App ID: ", app_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.projectName.value -o tsv
ml_project_name = deployment_stdout.n
print("👉🏻 Project Name: ", ml_project_name)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.projectId.value -o tsv
ml_project_id = deployment_stdout.n
print("👉🏻 Project Id: ", ml_project_id)

deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.containerAppResourceName.value -o tsv
containerapp_resource_name = deployment_stdout.n
print("👉🏻 Container App Name: ", containerapp_resource_name)

print(datetime.datetime.now().isoformat())


👉🏻 API Gateway URL:  https://apim-4hx6wwntb767o.azure-api.net
👉🏻 Workspace ID:  ddd7f56b-058e-47e2-8178-1dc1f319c76e
👉🏻 App ID:  787c9db5-b1e8-49df-b139-99e2e7424599
👉🏻 Project Name:  project-4hx6wwntb767o
👉🏻 Project Id:  /subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aip-pevo1-prompt-flow/providers/Microsoft.MachineLearningServices/workspaces/project-4hx6wwntb767o
👉🏻 Container App Name:  aca-4hx6wwntb767o
2024-10-25T11:39:50.635885


<a id='4'></a>
### 4️⃣ Enable Prompt Flow tracing (optional)

Prompt flow [tracing feature](https://microsoft.github.io/promptflow/how-to-guides/tracing/index.html) enables users to trace LLM calls, functions and even LLM frameworks. Besides, with promptflow[azure] installed, prompt flow can also log traces to an Azure ML workspace or Azure AI project, which makes it possible to share traces with your team members.

In [10]:
! pf config set trace.destination=azureml:/{ml_project_id}

The workspace Cosmos DB is not initialized yet, will start initialization, which may take some minutes...
waiting for Cosmos DB setup ready, current status: InProgress
waiting for Cosmos DB setup ready, current status: InProgress
waiting for Cosmos DB setup ready, current status: InProgress
waiting for Cosmos DB setup ready, current status: InProgress
waiting for Cosmos DB setup ready, current status: InProgress
waiting for Cosmos DB setup ready, current status: InProgress
waiting for Cosmos DB setup ready, current status: Succeeded
Set config [{'trace.destination': 'azureml://subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aip-pevo1-prompt-flow/providers/Microsoft.MachineLearningServices/workspaces/project-4hx6wwntb767o'}] successfully.


[2024-10-25 11:46:36 +0200][promptflow.azure._restclient.flow_service_caller][INFO] - start polling until Cosmos DB setup finished...
[2024-10-25 11:50:10 +0200][promptflow.azure._restclient.flow_service_caller][INFO] - Cosmos DB setup finished with status Succeeded


<a id='5'></a>
### 5️⃣ Update the connection with APIM Gateway URL and APIM Subscription Key

Notes:
- In this lab we will use the local basic-chat flow. Explore other sample flows from this [repo](https://github.com/microsoft/promptflow/tree/main/examples/flows/chat).
- Use the [VS Code Extension](https://marketplace.visualstudio.com/items?itemName=prompt-flow.prompt-flow) to build your own flows. Follow this [guide](https://microsoft.github.io/promptflow/how-to-guides/develop-a-dag-flow/index.html) to get started.

In [6]:
! pf connection create --file ./{flow_name}/azure_openai.yaml --set api_key={apim_subscription_key} api_base={apim_resource_gateway_url} --name open_ai_connection

{
    "auth_mode": "key",
    "name": "open_ai_connection",
    "module": "promptflow.connections",
    "created_date": "2024-10-25T11:40:49.909548",
    "last_modified_date": "2024-10-25T11:40:49.909548",
    "type": "azure_open_ai",
    "api_key": "******",
    "api_base": "https://apim-4hx6wwntb767o.azure-api.net",
    "api_type": "azure",
    "api_version": "2024-02-01"
}


<a id='localtest'></a>
### 🧪 Test the Flow locally


In [13]:
! pf flow test --flow {flow_name} --inputs question="What's OpenAI?"

Prompt flow service has started...
2024-10-25 11:56:52 +0200   34640 execution.flow     INFO     Start executing nodes in thread pool mode.
2024-10-25 11:56:52 +0200   34640 execution.flow     INFO     Start to run 1 nodes with concurrency level 16.
2024-10-25 11:56:52 +0200   34640 execution.flow     INFO     Executing node chat. node run id: 1d770249-d3be-4842-b48d-c4a434d81665_chat_0
2024-10-25 11:56:54 +0200   34640 execution.flow     INFO     Node chat completes.
You can view the trace detail from the following URL:
http://127.0.0.1:23333/v1.0/ui/traces/?#collection=basic-chat&uiTraceId=0xb161827928ab7e531cbc37c3ebd45abc
https://ai.azure.com/projecttrace/detail/0xb161827928ab7e531cbc37c3ebd45abc?wsid=/subscriptions/a46c2e68-1def-41bf-b7dd-c8ec765e3367/resourceGroups/aip-pevo1-prompt-flow/providers/Microsoft.MachineLearningServices/workspaces/project-4hx6wwntb767o
{
    "answer": "OpenAI is an artificial intelligence research organization that aims to ensure that artificial general

<a id='6'></a>
### 6️⃣ Create the Flow in the AI Studio Project (optional)

This feature is useful for sharing the flow with other users and leveraging the capabilities of Azure AI Studio. Open the [docs](https://microsoft.github.io/promptflow/cloud/azureai/manage-flows.html#create-a-flow) to learn more.

In [15]:
! pfazure flow create --flow {flow_name} --set display_name={flow_name} type=chat description="Basic Chat Flow" -g {resource_group_name} -w {ml_project_name}


Traceback (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\.venv\Scripts\pfazure.exe\__main__.py", line 7, in <module>
  File "c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\.venv\Lib\site-packages\promptflow\azure\_cli\entry.py", line 134, in main
    entry(command_args)
  File "c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\.venv\Lib\site-packages\promptflow\azure\_cli\entry.py", line 115, in entry
    cli_exception_and_telemetry_handler(run_command, activity_name, custom_dimensions)(args)
  File "c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\.venv\Lib\site-packages\promptflow\_cli\_utils.py", line 276, in wrapper
    raise e
  File "c:\Users\u3184\azdevops\pevosdcgh\AI-Gateway\.venv\Lib\site-packages\promptflow\_cli\_utils.py", line 264, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\u3184\azdevops\pevosdcgh\A

<a id='7'></a>
### 7️⃣ Build the Flow in the Docker format

This step will generate a build folder with a Dockerfile and prompt flow artifacts. Open the [docs](https://microsoft.github.io/promptflow/how-to-guides/deploy-a-flow/deploy-using-docker.html) to learn more.


In [ ]:
! pf flow build --source {flow_name} --output build --format docker

<a id='8'></a>
### 8️⃣ Deploy the Flow to an Azure Container App

Note ⚠️: If the command fails inside this notebook, please run it in a separate terminal.


In [ ]:
! echo az containerapp up --name {containerapp_resource_name} --source build --browse
! az containerapp up --name {containerapp_resource_name} --source build

<a id='requests'></a>
### 🧪 Test the Flow through APIM


In [ ]:
import uuid
UUID = str(uuid.uuid4())
print(f"Request-Id: {UUID} - use this ID to trace the requests in Azure Application Insights.")

response = requests.post(apim_resource_gateway_url + "/" + flow_name + "/score", headers = {'api-key':apim_subscription_key, "Request-Id": UUID}, json = {"question": "Which is the biggest football club in Portugal?"})
if (response.status_code == 200):
    data = json.loads(response.text)
    print("💬 ", data.get("answer"))
else:
    print(response.text)


<a id='portal'></a>
### 🔍 Analyze Application Insights transactions

Open the Application Insights resource in the Azure Portal and search for the Request-Id generated in the previous step.  
You can also search for the prompt flow execution, although end-to-end correlation is not yet available.

![result](result.png)


<a id='clean'></a>
### 🗑️ Clean up resources

When you're finished with the lab, you should remove all your deployed resources from Azure to avoid extra charges and keep your Azure subscription uncluttered.
Use the [clean-up-resources notebook](clean-up-resources.ipynb) for that.